In [1]:
!pip install -q pandas

In [1]:
!pip install -qU pip awscli boto3 sagemaker 
!pip install nvidia-pyindex
!pip install tritonclient[http]

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com


In [2]:
import os

In [3]:
import boto3
import json,
import sagemaker
import time
import os
from sagemaker import get_execution_role

sess = boto3.Session()
sm = sess.client("sagemaker")
sagemaker_session = sagemaker.Session(boto_session=sess)
role = get_execution_role()
client = boto3.client("sagemaker-runtime")

SyntaxError: invalid syntax (2559600829.py, line 2)

In [ ]:
triton_image_uri = "354625738399.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tritonserver:22.05-py"

## Package model files and uploading to S3

In [5]:
!wget -q -P triton-serve-fil/preprocessing https://rapidsai-data.s3.us-east-2.amazonaws.com/conda-pack/rapidsai/rapids22.06_cuda11.5_py3.8.tar.gz

In [ ]:
!tar -C triton-serve-fil/ -czf model.tar.gz preprocessing
model_uri = sagemaker_session.upload_data(path="model.tar.gz", key_prefix="triton-serve-fil")

In [4]:
def generate_config(model_dir, max_batch_size, num_features, threshold=0.5, 
                    deployment_type='gpu', storage_type='AUTO'):
    
    if deployment_type.lower() == 'cpu':
        instance_kind = 'KIND_CPU'
    else:
        instance_kind = 'KIND_GPU'

    config_text = f"""backend: "fil"
max_batch_size: {max_batch_size}
input [                                 
 {{  
    name: "input__0"
    data_type: TYPE_FP32
    dims: [ {num_features} ]                    
  }} 
]
output [
 {{
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }}
]
instance_group [{{ kind: {instance_kind} }}]
parameters [
  {{
    key: "model_type"
    value: {{ string_value: "xgboost_json" }}
  }},
  {{
    key: "output_class"
    value: {{ string_value: "true" }}
  }},
  {{
    key: "threshold"
    value: {{ string_value: "{threshold}" }}
  }},
  {{
    key: "storage_type"
    value: {{ string_value: "{storage_type}" }}
  }}
]

dynamic_batching {{
}}"""
    
    config_path = os.path.join(model_dir, 'config.pbtxt')
    with open(config_path, 'w') as file_:
        file_.write(config_text)

    return config_path

In [7]:
MAX_MEMORY_BYTES = 60_000_000
num_features = 15
num_classes = 2
bytes_per_sample = (num_features + num_classes) * 4
max_batch_size = MAX_MEMORY_BYTES // bytes_per_sample
deployment_type = 'gpu'
xgb_model_dir = 'xgboost'

In [8]:
generate_config(xgb_model_dir, max_batch_size, num_features, num_classes, deployment_type)

'xgboost/config.pbtxt'

In [1]:
import triton_python_backend_utils as pb_utils

ModuleNotFoundError: No module named 'triton_python_backend_utils'

## Run Inference

In [2]:
import pandas as pd

In [3]:
X_infer = pd.read_csv("X_infer.csv")

In [4]:
X_infer.head()

,User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Hour,Minute
0,767,0,2012,11,12,25.370001,1,22776,8313,145,26128,35,0,9,15
1,1676,0,2016,1,2,103.750000,0,61835,5977,28,23436,74,0,5,12
2,1395,0,2017,5,11,6.700000,0,21061,11313,214,25784,78,0,7,57
3,1073,1,2010,8,24,2.850000,2,39706,5783,128,4953,56,0,8,5
4,123,0,2009,12,17,80.779999,2,51879,6749,215,13903,54,0,13,7


In [6]:
X_infer.columns.tolist()

['User',
 'Card',
 'Year',
 'Month',
 'Day',
 'Amount',
 'Use Chip',
 'Merchant Name',
 'Merchant City',
 'Merchant State',
 'Zip',
 'MCC',
 'Errors?',
 'Hour',
 'Minute']

In [14]:
sample_data = X_infer.astype(str).values

In [15]:
sample_data

array([['767', '0', '2012', '11', '12', '25.37000084', '1', '22776',
        '8313', '145', '26128', '35', '0', '9', '15'],
       ['1676', '0', '2016', '1', '2', '103.75', '0', '61835', '5977',
        '28', '23436', '74', '0', '5', '12'],
       ['1395', '0', '2017', '5', '11', '6.699999809', '0', '21061',
        '11313', '214', '25784', '78', '0', '7', '57'],
       ['1073', '1', '2010', '8', '24', '2.849999905', '2', '39706',
        '5783', '128', '4953', '56', '0', '8', '5'],
       ['123', '0', '2009', '12', '17', '80.77999878', '2', '51879',
        '6749', '215', '13903', '54', '0', '13', '7'],
       ['1945', '2', '2014', '9', '28', '21.37000084', '2', '21752',
        '7688', '78', '16490', '95', '1', '16', '32'],
       ['579', '1', '2007', '9', '13', '11.39999962', '2', '56561',
        '7771', '107', '12881', '56', '0', '13', '6'],
       ['413', '3', '2010', '9', '7', '55.43000031', '2', '8910', '3283',
        '214', '25904', '65', '0', '7', '58'],
       ['83', '0', '

In [18]:
type(sample_data)

numpy.ndarray

In [ ]:
payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": list(sample_data.shape), "datatype": "STRING", "data": sample_data},
    ]
}